In [1]:
# Nhập thư viện
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import time

import warnings
warnings.filterwarnings('ignore')

# Đọc bộ dữ liệu
corpus = pd.read_csv('data/cleaned_mhc.csv')

In [2]:
# Chuẩn bị dữ liệu
X_train, X_test, y_train, y_test = train_test_split(
    corpus['text'],
    corpus['label'],
    test_size=0.2,
    random_state=42
)

tfidf = TfidfVectorizer(max_features=3500)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)

n_components = 100

lsa = TruncatedSVD(n_components=n_components, random_state=42)
X_train_lsa = lsa.fit_transform(X_train_tfidf)
X_test_lsa = lsa.transform(X_test_tfidf)

print(f"TF-IDF - Train shape: {X_train_tfidf.shape}, Test Shape: {X_test_tfidf.shape}")
print(f"LSA - Train shape: {X_train_lsa.shape}, Test Shape: {X_test_lsa.shape}")

TF-IDF - Train shape: (18592, 3500), Test Shape: (4648, 3500)
LSA - Train shape: (18592, 100), Test Shape: (4648, 100)


In [3]:
k_values = [1, 5, 10, 50, 100, 200, 500, 1000, 2000, 4000]
metrics = ['euclidean', 'cosine']
datasets = {'TF-IDF': (X_train_tfidf, X_test_tfidf), 'LSA': (X_train_lsa, X_test_lsa)}

results = pd.DataFrame(columns=['Dataset', 'Distance', 'K', 'Train Accuracy', 'Test Accuracy',
                                'F1 Score', 'Precision', 'Recall', 'Test Time (s)'])

for dataset_name, (X_train, X_test) in datasets.items():
    for metric in metrics:
        for k in k_values:
            knn = KNeighborsClassifier(n_neighbors=k, metric=metric)
            
            knn.fit(X_train, y_train)
            start_time = time.time()
            y_pred = knn.predict(X_test)
            test_time = time.time() - start_time
            
            train_accuracy = knn.score(X_train, y_train)
            test_accuracy = accuracy_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred, average='weighted')
            precision = precision_score(y_test, y_pred, average='weighted')
            recall = recall_score(y_test, y_pred, average='weighted')
            
            results.loc[len(results)] = [dataset_name, metric, k, train_accuracy, test_accuracy, 
                                         f1, precision, recall, test_time]
            
            print(f"Finished KNN for {dataset_name} with {metric} distance at K={k}")

best_models = results.loc[results.groupby(['Dataset', 'Distance'])['F1 Score'].idxmax()]

for _, row in best_models.iterrows():
    best_knn = KNeighborsClassifier(n_neighbors=int(row['K']), metric=row['Distance'])
    best_knn.fit(datasets[row['Dataset']][0], y_train)
    y_best_pred = best_knn.predict(datasets[row['Dataset']][1])
    confusion = confusion_matrix(y_test, y_best_pred)
    
    print(f"\nBest model for {row['Dataset']} with {row['Distance']} distance and K={row['K']}:")
    print(f"Train Accuracy: {row['Train Accuracy']:.4f}, Test Accuracy: {row['Test Accuracy']:.4f}")
    print(f"F1 Score: {row['F1 Score']:.4f}, Precision: {row['Precision']:.4f}, Recall: {row['Recall']:.4f}")
    print(f"Test Time: {row['Test Time (s)']:.4f} seconds")
    print("Confusion Matrix:\n", confusion)

Finished KNN for TF-IDF with euclidean distance at K=1
Finished KNN for TF-IDF with euclidean distance at K=5
Finished KNN for TF-IDF with euclidean distance at K=10
Finished KNN for TF-IDF with euclidean distance at K=50
Finished KNN for TF-IDF with euclidean distance at K=100
Finished KNN for TF-IDF with euclidean distance at K=200
Finished KNN for TF-IDF with euclidean distance at K=500
Finished KNN for TF-IDF with euclidean distance at K=1000
Finished KNN for TF-IDF with euclidean distance at K=2000
Finished KNN for TF-IDF with euclidean distance at K=4000
Finished KNN for TF-IDF with cosine distance at K=1
Finished KNN for TF-IDF with cosine distance at K=5
Finished KNN for TF-IDF with cosine distance at K=10
Finished KNN for TF-IDF with cosine distance at K=50
Finished KNN for TF-IDF with cosine distance at K=100
Finished KNN for TF-IDF with cosine distance at K=200
Finished KNN for TF-IDF with cosine distance at K=500
Finished KNN for TF-IDF with cosine distance at K=1000
Finish

$
\begin{bmatrix}
\text{True Negative (TN) - Class 0} & \text{False Positive (FP) - Predict 1 but 0} \\
\text{False Negative (FN) - Predict 0 but 1} & \text{True Positive (TP) - Class 1} \\
\end{bmatrix}
$

In [4]:
tfidf_table = results[results["Dataset"] == "TF-IDF"].pivot(index="K", columns="Distance", values=[
    "Train Accuracy", "Test Accuracy", "F1 Score", "Precision", "Recall", "Test Time (s)"
])

lsa_table = results[results["Dataset"] == "LSA"].pivot(index="K", columns="Distance", values=[
    "Train Accuracy", "Test Accuracy", "F1 Score", "Precision", "Recall", "Test Time (s)"
])

def apply_dual_heatmap(styler):
    euclidean_columns = [col for col in styler.columns if col[1] == 'euclidean']
    styler = styler.background_gradient(subset=euclidean_columns, cmap='Reds')
    
    cosine_columns = [col for col in styler.columns if col[1] == 'cosine']
    styler = styler.background_gradient(subset=cosine_columns, cmap='Blues')
    
    return styler

tfidf_table_styled = tfidf_table.style.pipe(apply_dual_heatmap)
lsa_table_styled = lsa_table.style.pipe(apply_dual_heatmap)

print("TF-IDF Table with Dual Heatmap:")
display(tfidf_table_styled)

print("\nLSA Table with Dual Heatmap:")
display(lsa_table_styled)

TF-IDF Table with Dual Heatmap:



LSA Table with Dual Heatmap:
